# Lab de MongoDB - Introduccion

Las fuentes y recursos de este laboratorio provienen de 
- [mongodb-labs](https://github.com/mattdavis0351/mongodb-labs) por **Ishana Raina** y **Matthew Davis**

## Outline del Lab 1

- De un JSON, crear la base de datos al MongoDB Server por medio de MongoDB Database Tools
- Visualizar los datos de la base de datos
- Anhadir datos y entender su funcionamiento

## Crear base de datos por medio de JSON

Para poder cargar los datos de un JSON hacia MongoDB creando una base de datos, se tiene que usar el ejecutable `mongoimport`, este ejecutable(que es una instalacion aparte de MongoDB Server) nos pide:
<pre>
- --db          Nombre de la base de datos
- --collection  Nombre de la coleccion(tabla)
- --file        Localizacion del archivo
- --jsonArray   Indica que el JSON a importar es un arreglo
</pre>

Y con esto podremos cargar la base de datos a MongoDB

In [ ]:
mongoimport --db sample_mflix --collection movies --file "C:\Users<completar el path con el movies.json en la carpeta data>" --jsonArray

Para eliminar una base de datos hay que realizar lo siguiente
```bat
:: Ingrear a mongosh para manipular la base de datos
mongosh

:: Mostrar las bases de datos existentes
show databases

:: Seleccionar la base de datos a borrar
use <nombreBD>

:: Borrar la base de datos
db.dropDatabase()
```

## Pymongo

Para poder usar MongoDB en python, hay que descargar por medio de pip `pymongo`, driver para poder aceder al API de MongoDB y manipular la base de datos

Para instalarlo hay que ejecutar `pip install pymongo`, y se podra acceder a MongoDB desde python

In [ ]:
pip install pymongo

# Setup

Ahora con el driver instalado, hay que importarlo y usarlo

Recordar tener `mongod` corriendo en una terminal para que funcione

In [ ]:
from pymongo import MongoClient
client = MongoClient()

# Confirmar la conexion a MongoDB
client.server_info()

# Conseguir los datos cargados

Ahora para poder conseguir los datos de una coleccion(`db.movies`), podemos usar distintas funciones para ello, como las siguientes:
<pre>
- find_one()                Recupera el primer documento dentro de la coleccion
- find({})                  Recupera varios documentos solo si cumplen con el criterio dado(vacio recupera todos)
- distinct('')              Recupera una sola vez los valores distintos segun la llave especificada
                            filtro especificado(si esta vacio entonces recupera todos los documentos)
- find({}, {'filter'})      Lo mismo que "find()" pero con la capacidad de mostrar 
                            los documentos filtrando las llaves que querramos de los documentos
</pre>

In [87]:
# Setup de las variables a usar
db = client.sample_mflix
moviesCollection = db.movies

In [ ]:
# - find_one()                Recupera el primer documento dentro de la coleccion

result = db.movies.find_one()

result

In [ ]:
# - find({})                  Recupera multiples documentos solo si cumplen con el 

db = client.sample_mflix
moviesCollection = db.movies
allMovies = list(moviesCollection.find({}))

print('Cantidad de pelicluas cargadas en la coleccion: ', len(allMovies))
# Mostrando solo la primera pelicula de todas las que hay
allMovies[0]

In [ ]:
# - distinct('')              Recupera una sola vez los valores distintos segun la llave especificada
#                             filtro especificado(si esta vacio entonces recupera todos los documentos)

# Usar distinct con la llave 'runtime' para ver todos los distintos runtimes existentes dentro de este dataset de peliculas
uniqueNames = moviesCollection.distinct('runtime')
print(uniqueNames)

In [ ]:
# - find({}, {'filter'})      Lo mismo que "find()" pero con la capacidad de mostrar 
#                             los documentos filtrando las llaves que querramos de los documentos

# Recuperando todos los documentos pero solo con las llaves '"title", "imdb", "year", "plot"', e ignorando todos los demas
# Util para poder ver documentos con las llaves que nos interesen y evitar ver todas la propiedades que no nos importan
allMovies = list(moviesCollection.find({}, {"title", "imdb", "year", "plot"}))
allMovies[0]

# Conseguir y operar sobre los datos cargados

Ademas de tener varias formas de conseguir los datos como tal, tambien podemos **realizar operaciones** sobre los mismos que nos devuelvan los mismos datos, pero centrandonos en transformar estos datos en formatos mas precisos
<pre>
- find({}, {'operator'})    Recupera los documentos que cumplan con un criterio de los operadores usados
- find({}).sort("llave", 1) El 'sort' permite ordenar los documentos conseguidos segun la 'llave' sobre
                            la que querramos ordenar y la forma de ordenar                            
- aggregate(pipeline)       Recupera los documentos segun operaciones especificadas como pares 
                            llave-valor dentro del pipeline y ejecuta las operaciones en los documentos 
</pre>

In [ ]:
# - find({}, {'operator'})    Recupera los documentos que cumplan con un criterio de los operadores usados 

# Usando el filtro con el operador 'lt' para encontrar peliculas antes del anho 2011
moviesFrom2011 = list(moviesCollection.find({'year': {'$lt': 2011}}, {"title"}))
moviesFrom2011

In [ ]:
# - find({}, {'operator'})    Recupera los documentos que cumplan con un criterio de los operadores usados 

# Filtro con operadores para conseguir las peliculas con runtime menor a 3 o mayor a 600
filter = {
    "$or": [
        {
            "runtime": {
                "$lt":3
            }
        },
        {
            "runtime": {
                "$gt": 600
            }
        }
    ]
}

moviesWithSmallOrBigRuntime = list(db.movies.find(filter, {'title', 'runtime'}))
moviesWithSmallOrBigRuntime

In [ ]:
# - find({}).sort("llave", 1) El 'sort' permite ordenar los documentos conseguidos segun la 'llave' sobre
#                             la que querramos ordenar y la forma de ordenar                            

# Recuperar las peliculas del anho 2011, filtrar las llaves 'title' y 'awards', y ordenar las peliculas 
# de mayor a menor las peliculas segun la cantidad de premios ganados
# Top 7 mejores peliculas del 2011
topBestMovies = list(db.movies.find({"year": 2011 }, {'title', 'awards.wins'}).sort("awards.wins", -1).limit(7))

topBestMovies


In [ ]:
# - aggregate(pipeline)       Recupera los documentos segun operaciones especificadas como pares 
#                             llave-valor dentro del pipeline y ejecuta las operaciones en los documentos 

# Agrupar las peliculas segun su anho de salida, y contar los elementos de cada grupo(o sea contar cuantas peliculas salieron cada anho)
pipeline = [
    {
        '$group': {
            '_id': '$year', # Criterio para el agrupamiento de una llave
            'count': {'$sum': 1} # Operacion de agrupamiento(mismo formato de los demas operadores del API)
        }
    }
]

result = moviesCollection.aggregate(pipeline)
for item in result:
    print(f"Release year: {item['_id']}\t Count: {item['count']}")

In [ ]:
# - aggregate(pipeline)       Recupera los documentos segun operaciones especificadas como pares 
#                             llave-valor dentro del pipeline y ejecuta las operaciones en los documentos 
# Uso de la funcion 'aggregate' para conseguir la pelicula que mas premios ha ganado en 2014

# En las siguinetes 3 variables, se crearan los operadores que luego se uniran por medio del 'pipeline' 
# y ejecutaran los 3 operadores sobre los datos
stage_match_year = { 
    "$match": {"year": 2014}
}
stage_sort_awards_desc = {
    "$sort" : {"awards.wins": -1}
}
stage_limit_1 = {
    "$limit" : 1
}
pipeline = [
    stage_match_year,
    stage_sort_awards_desc,
    stage_limit_1,
]

results = db.movies.aggregate(pipeline)
resultsList = []

for movie in results:
    resultsList.append(movie)

resultsList